In [19]:
!pip install nltk

In [20]:
import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
import json
import unicodedata

def remove_diacritics_and_lower(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    without_diacritics = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    return without_diacritics.lower()

# Đọc nội dung từ tệp intents.json
with open('/content/sample_data/intents.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Loại bỏ dấu diacritics và chuyển đổi thành chữ thường từ mỗi chuỗi trong dữ liệu
for intent in data['intents']:
    for key in intent:
        if isinstance(intent[key], list):
            for i, item in enumerate(intent[key]):
                intent[key][i] = remove_diacritics_and_lower(item)
        else:
            intent[key] = remove_diacritics_and_lower(intent[key])

# Ghi lại nội dung đã thay đổi vào tệp
with open('intents.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


In [22]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [23]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    if len(bag) > len(output_row):
        output_row.extend([0] * (len(bag) - len(output_row)))
    elif len(bag) < len(output_row):
        bag.extend([0] * (len(output_row) - len(bag)))
    training.append([bag, output_row])

random.shuffle(training)
# Check the lengths of 'bag' and 'output_row' lists
print(len(bag))
print(len(output_row))


# Convert the 'training' list to a NumPy array
training = np.array(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

275
275


In [24]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Use SGD with the learning rate schedule
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)

print('Done')

Epoch 1/200
21/21 [==============================] - 1s 4ms/step - loss: 5.5930 - accuracy: 0.0000e+00
Epoch 2/200
21/21 [==============================] - 0s 3ms/step - loss: 5.4920 - accuracy: 0.0485
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 5.3252 - accuracy: 0.1068
Epoch 4/200
21/21 [==============================] - 0s 4ms/step - loss: 5.1276 - accuracy: 0.0388
Epoch 5/200
21/21 [==============================] - 0s 4ms/step - loss: 4.6747 - accuracy: 0.1068
Epoch 6/200
21/21 [==============================] - 0s 4ms/step - loss: 4.3374 - accuracy: 0.0583
Epoch 7/200
21/21 [==============================] - 0s 4ms/step - loss: 3.9717 - accuracy: 0.0583
Epoch 8/200
21/21 [==============================] - 0s 4ms/step - loss: 3.6380 - accuracy: 0.1068
Epoch 9/200
21/21 [==============================] - 0s 4ms/step - loss: 3.7002 - accuracy: 0.0777
Epoch 10/200
21/21 [==============================] - 0s 4ms/step - loss: 3.6827 - accuracy: 0.0874
Epoch

In [25]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

In [27]:
intents = json.loads(open('/content/intents.json').read())
words = pickle.load(open('/content/words.pkl', 'rb'))
classes = pickle.load(open('/content/classes.pkl', 'rb'))
model = load_model('/content/chatbotmodel.h5')


In [28]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [30]:
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        print("|===================== The Program End here! =====================|")
        exit()

    else:
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)


| You: xin chào
1/1 [==============================] - 0s 18ms/step
| Bot: xin chao, rat vui duoc gap lai ban
| You: ban bao nhieu tuoi 
1/1 [==============================] - 0s 17ms/step
| Bot: hai muoi mot tuoi
| You: thu ba 26/3/2024 co su kien gi
1/1 [==============================] - 0s 18ms/step
| Bot: tu toi 25/3, bitcoin kich hoat đa tang sau thoi gian dai giao dich quanh vung 64.000-68.000 usd mot đon vi. khoang 23h45, tien so lon nhat the gioi vuot 70.000 usd, tang hon 7% trong 24 gio.
thi gia co chieu huong đi len, đen sang 26/3, bitcoin co luc đat hon 71.162 usd. trong buoi sang, thi truong co rung lac nhung tien so nay van giu đuoc muc cao nhat 10 ngay qua. luc 9h30, moi đong bitcoin khoang 70.238 usd, cao hon nguong đinh thiet lap thang 11/2021 (68.818 usd).
cac tien so khac cung dien bien tich cuc. ether đa tang 6%, solana va avalanche cung tich luy hon 10% trong 24 gio.
viec bitcoin lay lai moc 70.000 usd bao hieu đot đieu chinh co the ket thuc. ve đa tang gia lan nay,

KeyboardInterrupt: Interrupted by user